# Understanding variables

Can the data answer the question?

-- DS: optimise customer experience by tailoring cnotent to user needs (identify behaviour patterns)

-- ML: difficulty finding content (identify 404 patterns) (identify opportunity to improve)

In [1]:
import pandas as pd
import numpy as np
import os
import re

## Load data

In [2]:
# constants
folder_raw_data = '../data/raw'
folder_clean_data = '../data/cleaned'
symbols_dirty = re.compile(' -0400| -|\[|\]')
raw_encoding='iso-8859-1'

# variables
cols_raw_log = ['host', 'timestamp', 'request', 'response', 'bytes']
dtype_cols_raw_log = {'host': str, 'timestamp': np.datetime64, 'request': str, 'response': str, 'bytes': np.int64}

Need to clean data from unnecessary dashes so the tsv parsing would be more straightforward.

In [3]:
def clean_raw_logs(file):
    with open(folder_clean_data + '/' + file, 'w', encoding=raw_encoding) as c:
        with open(folder_raw_data + '/' + file, 'r', encoding=raw_encoding) as r:
            for log_line in r:
                log_line = symbols_dirty.sub('', log_line)
                c.write(log_line)

Ensuring adherence to data type

In [4]:
def apply_data_type(df_logs):
    df_logs['timestamp'] = pd.to_datetime(df_logs['timestamp'], format='%d/%b/%Y:%H:%M:%S')
    df_logs['bytes'] = df_logs['bytes'].fillna(0)
    df_logs = df_logs.astype(dtype_cols_raw_log)
    return df_logs

Loading the raw logs and converting in an expected dataframe output

In [5]:
def load_logs():
    files = os.listdir(folder_raw_data)
    files = [file for file in files if os.path.isfile(folder_raw_data + '/' + file)]
    df_logs = pd.DataFrame(columns=cols_raw_log)
    for file in files:
        clean_raw_logs(file)
        df_log = pd.read_csv(folder_clean_data + '/' + file, sep=' ', encoding=raw_encoding, header=None, names=cols_raw_log, 
                             on_bad_lines='skip') #'warn')
        df_log = apply_data_type(df_log)
        df_logs = pd.concat([df_logs, df_log], axis=0)
    return df_logs

df_loaded_logs = load_logs()


/var/folders/xr/9y_qxq8j3mzgvbyy734y4_pc0000gn/T/ipykernel_38959/2534735201.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_log = pd.read_csv(folder_clean_data + '/' + file, sep=' ', encoding=raw_encoding, header=None, names=cols_raw_log,
/var/folders/xr/9y_qxq8j3mzgvbyy734y4_pc0000gn/T/ipykernel_38959/2534735201.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_log = pd.read_csv(folder_clean_data + '/' + file, sep=' ', encoding=raw_encoding, header=None, names=cols_raw_log,


In [6]:
df_loaded_logs.head()

,host,timestamp,request,response,bytes
0,199.72.81.55,1995-07-01 00:00:01,GET /history/apollo/ HTTP/1.0,200,6245
1,unicomp6.unicomp.net,1995-07-01 00:00:06,GET /shuttle/countdown/ HTTP/1.0,200,3985
2,199.120.110.21,1995-07-01 00:00:09,GET /shuttle/missions/sts-73/mission-sts-73.ht...,200,4085
3,burger.letters.com,1995-07-01 00:00:11,GET /shuttle/countdown/liftoff.html HTTP/1.0,304,0
4,199.120.110.21,1995-07-01 00:00:11,GET /shuttle/missions/sts-73/sts-73-patch-smal...,200,4179


In [7]:
df_loaded_logs.dtypes

host         object
timestamp    object
request      object
response     object
bytes        object
dtype: object

# Clean dataset

-- refactor URL

-- variable selection (create new variable indicators)

-- removing rows with null values

-- identify outliers?

# Identify relationship between variables